<a href="https://colab.research.google.com/github/gl7176/CNN_tools/blob/main/training_from_multiple_sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import tiles from one dataset and run a model from another dataset

**Before running this script, enter the drive folder links of each dataset you would like to use to train the model. Each drive folder should include (1) tiled images, (2) the `tiling_scheme.json` file, and (3) the training data associated with each tile set**

<a href="https://colab.research.google.com/github/gl7176/CNN_local/blob/main/Train_from_multiple_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
#####  <center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

In [ ]:
# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1DKAp-k2cHWFj9rLNhNL4i6dKoPcR3Gn6'

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls every file in the directory specified above
image_list = []
count = 0
for f in file_list:
  fname = os.path.join(local_download_path, f['title'])
  if fname.endswith(".png"):
    image_list.append(fname)
    count += 1
    if count % 10 == 0:
      print(str(count) + " tiles pulled")
  # 3. Create & download by id.
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  if fname.endswith(".h5"):
    inferential_model = fname
    print("Pulled model: " + fname)
  if fname.endswith(".csv"):
    if fname.endswith("classes.csv"):
      classes_file = fname
      print("Pulled classes file")
  if fname.endswith(".json"):
    if fname.endswith("tiling_scheme.json"):
      tiling_scheme_file = fname
      print("Pulled tiling scheme file")
print(str(count) + " tiles pulled")

Pulled classes file
Pulled tiling scheme file
Pulled model: data/HayIsland2015_b2_e18_inference.h5
10 tiles pulled
20 tiles pulled
30 tiles pulled
40 tiles pulled
50 tiles pulled
60 tiles pulled
70 tiles pulled
80 tiles pulled
90 tiles pulled
100 tiles pulled
110 tiles pulled
120 tiles pulled
130 tiles pulled
134 tiles pulled
